## Machine Learning Engineer Nanodegree
### Capstone Project
### Project: Toxic Comment Classification with CNN

### Solution Implementation

In this step, we did the following:

    - train the final model selected
    - run the prediction on the unseen test dataset
    - plot the roc curves
    - plot roc_auc score vs epoch
    - plot loss vs epoch
    - calculate roc_auc score for changes in test dataset to see impact of input perturbations

The CNN model has the following parameters:

    - regions: [2,3,4]
    - number of filters: 32
    - spatial dropout rate: 0.4
    - dropout rate: 0.2
    - activation function: tanh
    - epochs: 25
    - batch size: 256
    - no stop words removing
    - not removing IPs, usernames and URL links
    - validation dataset: 10%
    - optimizer: adadelta(lr=1.0,rho=0.95,epsilon=None,decay=0.0)
    - loss function: binary_crossentropy
    - metrics: tf_binary_auc, sklearn roc_auc

The default parameters are used for Keras tokenizer:

    - (numwords=None, filters='!"#$%&()*+,-./:;<=>?@[]^`{|}~ ', lower=True, split=' ', char_level=False, oov_token=None, document_count=0)

The following input files are located in the same directory as this notebook.

    1. train.csv - the training set, contains comments with their binary labels
    2. test.csv - the test set, you must predict the toxicity probabilities for these comments. To deter hand labeling, the test set contains some comments which are not included in scoring.
    3. test_labels.csv - labels for the test data; value of -1 indicates it was not used for scoring; (Note: file added after competition close!)
    4. crawl-300d-2M.vec - the pre-trained word vectors

Output files

    1. The best model is saved as hd5f file at the location pointed by file_path variable.
    2. The prediction results are saved as csv file at the location pointed by save_path variable.

In [2]:


# Import the required classes
import numpy as np
import pandas as pd
np.random.seed(8)
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from keras.models import Model
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, Conv1D, MaxPooling1D, GlobalMaxPooling1D 
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.preprocessing import text, sequence
from keras.callbacks import Callback

import re
import warnings
warnings.filterwarnings('ignore')

# Read in the train and test data
train = pd.read_csv('/opt/data/share01/jl2408/train.csv')
test_cm = pd.read_csv('/opt/data/share01/jl2408/test.csv')
test_lb = pd.read_csv('/opt/data/share01/jl2408/test_labels.csv')
# Merge test comments with the labels
test_all = pd.merge(test_cm, test_lb, on='id')
# Create test dataset by removing samples not used for scoring in Kaggle
test = test_all[test_all['toxic'] != -1]



In [3]:
# Create features and labels for train and test
label_names = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
X_train = train["comment_text"]
y_train = train[label_names].values
X_test = test["comment_text"]
y_test = test[label_names].values

In [4]:
# Define vocabulary size
vocab = 100000
# Define maximum length of a comment
maxlen = 200
# Define embedding size which should equal the embedding size of the pre-trained word vectors
embed_size = 300

In [5]:
# Tokenize the train dataset
t = text.Tokenizer(num_words=vocab)
t.fit_on_texts(list(X_train))

In [6]:
# Convert both train and test datasets into sequences
X_train = t.texts_to_sequences(X_train)
X_test = t.texts_to_sequences(X_test)
x_train = sequence.pad_sequences(X_train, maxlen=maxlen)
x_test = sequence.pad_sequences(X_test, maxlen=maxlen)

In [7]:
# Load pre-trained word vectors
EMBEDDING_FILE = '/opt/data/share01/jl2408/crawl-300d-2M.vec'
embeddings_index = dict()
f = open(EMBEDDING_FILE)
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s pre-trained words' % len(embeddings_index))

In [8]:
# Create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab, embed_size))
for word, i in t.word_index.items():
    if i >= vocab: 
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [9]:
# Import tensorflow class
import tensorflow as tf
# Import keras backend
import keras.backend as K

# Define a custom metric to calculate roc_auc based on tensorflow API
# FROM https://www.kaggle.com/c/porto-seguro-safe-driver-prediction/discussion/41108
'''
def jacek_auc(y_true, y_pred):
   score, up_opt = tf.metrics.auc(y_true, y_pred)
   #score, up_opt = tf.contrib.metrics.streaming_auc(y_pred, y_true)    
   K.get_session().run(tf.local_variables_initializer())
   with tf.control_dependencies([up_opt]):
       score = tf.identity(score)
   return score
'''

# Define a custom metric to calculate roc_auc based on tensorflow API
# FROM https://www.kaggle.com/c/porto-seguro-safe-driver-prediction/discussion/41015
# AUC for a binary classifier
def tf_binary_auc(y_true, y_pred):
    ptas = tf.stack([binary_PTA(y_true,y_pred,k) for k in np.linspace(0, 1, 1000)],axis=0)
    pfas = tf.stack([binary_PFA(y_true,y_pred,k) for k in np.linspace(0, 1, 1000)],axis=0)
    pfas = tf.concat([tf.ones((1,)) ,pfas],axis=0)
    binSizes = -(pfas[1:]-pfas[:-1])
    s = ptas*binSizes
    return K.sum(s, axis=0)

#---------------------
# PFA, prob false alert for binary classifier
def binary_PFA(y_true, y_pred, threshold=K.variable(value=0.8)):
    y_pred = K.cast(y_pred >= threshold, 'float32')
    # N = total number of negative labels
    N = K.sum(1 - y_true)
    # FP = total number of false alerts, alerts from the negative class labels
    FP = K.sum(y_pred - y_pred * y_true)
    return FP/N

#----------------
# P_TA prob true alerts for binary classifier
def binary_PTA(y_true, y_pred, threshold=K.variable(value=0.8)):
    y_pred = K.cast(y_pred >= threshold, 'float32')
    # P = total number of positive labels
    P = K.sum(y_true)
    # TP = total number of correct alerts, alerts from the positive class labels
    TP = K.sum(y_pred * y_true)
    return TP/P

In [10]:


# Create a class to calculate Sklearn roc_auc_score for both train and validation
# at the end of batch and epoch
from sklearn.metrics import roc_auc_score
import keras
class RocAucMetricCallback(Callback):
    def __init__(self, train_data, predict_batch_size=1024, include_on_batch=False):
        super(RocAucMetricCallback, self).__init__()
        self.train_data=train_data
        self.predict_batch_size=predict_batch_size
        self.include_on_batch=include_on_batch

    def on_batch_begin(self, batch, logs={}):
        pass

    def on_batch_end(self, batch, logs={}):
        if(self.include_on_batch):
            logs['sklearn_auc']=float('-inf')
            logs['val_sklearn_auc']=float('-inf')
            if(self.train_data):
                logs['sklearn_auc']=roc_auc_score(self.train_data[1], 
                                                  self.model.predict(self.train_data[0],
                                                                     batch_size=self.predict_batch_size))
            if(self.validation_data):
                logs['val_sklearn_auc']=roc_auc_score(self.validation_data[1], 
                                                  self.model.predict(self.validation_data[0],
                                                                     batch_size=self.predict_batch_size))

    def on_train_begin(self, logs={}):
        if not ('sklearn_auc' in self.params['metrics']):
            self.params['metrics'].append('sklearn_auc')
        if not ('val_sklearn_auc' in self.params['metrics']):
            self.params['metrics'].append('val_sklearn_auc')

    def on_train_end(self, logs={}):
        pass

    def on_epoch_begin(self, epoch, logs={}):
        pass

    def on_epoch_end(self, epoch, logs={}):
        logs['sklearn_auc']=float('-inf')
        logs['val_sklearn_auc']=float('-inf')
        if(self.train_data):
            logs['sklearn_auc']=roc_auc_score(self.train_data[1], 
                                              self.model.predict(self.train_data[0],
                                                                 batch_size=self.predict_batch_size))
            print("- sklearn_auc: %.4f" % (logs['sklearn_auc']))    
        if(self.validation_data):
            logs['val_sklearn_auc']=roc_auc_score(self.validation_data[1], 
                                              self.model.predict(self.validation_data[0],
                                                                 batch_size=self.predict_batch_size))
            print("- val_sklearn_auc: %.4f" % (logs['val_sklearn_auc']))    



In [11]:
# Define make_model function to create a CNN model
def make_model(k=[2,3,4], activation='tanh', filters=32, Sdroprate=0.4, droprate=0.2):

    # Input layer
    inp = Input(shape=(maxlen, ))
    # embedding layer
    x = Embedding(vocab, embed_size, weights=[embedding_matrix], input_length=maxlen, trainable=False)(inp)
    # Spatial dropout layer
    x = SpatialDropout1D(Sdroprate)(x)
    # Reshape to connect to 2D layer
    x = Reshape((maxlen, embed_size, 1))(x)
    
    # Convolutional and 1-max pooling layers
    conv = dict()
    maxpool = dict()
    for h in k:
        conv[h] = Conv2D(filters, kernel_size=(h, embed_size), activation=activation)(x)
        maxpool[h] = MaxPool2D(pool_size=(maxlen - h + 1, 1))(conv[h])
    if len(k) == 1:
        y = maxpool[h]
    else:
        # Concatenate multiple 1-max pools
        y = Concatenate(axis=1)([pool for key,pool in maxpool.items()])
    # Flatten layer
    y = Flatten()(y)
    # Dropout layer
    y = Dropout(droprate)(y)
    # Output dense layer
    outp = Dense(6, activation="sigmoid")(y)
    # Generate the cnn model
    model = Model(inputs=inp, outputs=outp)
    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer='adadelta', metrics=[tf_binary_auc])
    
    return model
# Make the model
model = make_model()
# Print out model summary
model.summary()

In [12]:


# Split the train dataset into train and validation datasets
x_trainS, x_val, y_trainS, y_val = train_test_split(x_train, y_train, train_size=0.90, random_state=8)



In [13]:
# Set batch_size and epochs parameter
batch_size=256
epochs = 25
# Define file_path to store best model
file_path = '/opt/data/share01/jl2408/e25.234.run5.hdf5'
# Create instance of ModelCheckpoint, EarlyStopping and RocAucMetricCallback
from keras.callbacks import ModelCheckpoint, EarlyStopping
check_point = ModelCheckpoint(filepath=file_path, monitor = 'val_sklearn_auc', mode="max", verbose=1, save_best_only=True)
early_stop = EarlyStopping(monitor = 'val_sklearn_auc', mode = "max", patience = 2, verbose=2)
roc_auc = RocAucMetricCallback(train_data=(x_trainS, y_trainS))
# Define callback
#cb = [roc_auc, early_stop, check_point]
cb = [roc_auc, check_point]
# Fit the model with the train and validation datasets
from keras.wrappers.scikit_learn import KerasClassifier
estimator = KerasClassifier(make_model,epochs=epochs,batch_size=batch_size,verbose=1)
h = estimator.fit(x_trainS, y_trainS, validation_data=(x_val, y_val), callbacks=cb)

In [14]:
# Load the saved best model and print out the model summary
from keras.models import load_model
model_saved = load_model(file_path, custom_objects={'tf_binary_auc': tf_binary_auc})
model_saved.summary()

In [15]:
# Run prediction with the saved model on unseen test dataset and calculate the roc_auc score
y_pred = model_saved.predict(x_test, batch_size=batch_size)
score = roc_auc_score(y_test, y_pred)
print("\n roc_auc score: %.6f \n" % (score))

In [16]:
# Compute ROC curves and AUCs for test
from sklearn.metrics import roc_curve, auc
from scipy import interp

# Compute false positive rate (fpr), true positive rate (tpr) and area under the curves (rocauc)
fpr = dict()
tpr = dict()
thr = dict()
rocauc = dict()
for i in range(y_test.shape[1]):
    fpr[i], tpr[i], thr[i] = roc_curve(y_test[:, i], y_pred[:, i])
    rocauc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], thr["micro"] = roc_curve(y_test.ravel(), y_pred.ravel())
rocauc["micro"] = auc(fpr["micro"], tpr["micro"])

# Compute Macro-average ROC curve and ROC area
# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(y_test.shape[1])]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(y_test.shape[1]):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= y_test.shape[1]

# Then interpolate all ROC curves at this points
mean_thr = np.zeros_like(all_fpr)
for i in range(y_test.shape[1]):
    mean_thr += interp(all_fpr, fpr[i], thr[i])

# Finally average it and compute AUC
mean_thr /= y_test.shape[1]

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
thr["macro"] = mean_thr
rocauc["macro"] = auc(fpr["macro"], tpr["macro"])

# to increase tpr, you also increase fpr. It is a trade off between the two.
# Business requirement determine which one is more important
# We can calculate the threshold to garantee a minimum tpr or a maximum fpr
# We see here that to increase tpr, we need to lower threshold
# to decrease fpr, we need to increase threshold

tpr_cut = 0.90
fpr_cut = 0.10

# index of the first threshold for which the sensitivity > tpr_cut
idx1 = np.min(np.where(tpr["micro"] > tpr_cut)) 
# index of the first threshold for which the fall out < fpr_cut
idx2 = np.max(np.where(fpr["micro"] < fpr_cut)) 
# index of the first threshold for which the sensitivity > tpr_cut
idx3 = np.min(np.where(tpr["macro"] > tpr_cut)) 
# index of the first threshold for which the fall out < fpr_cut
idx4 = np.max(np.where(fpr["macro"] < fpr_cut)) 
# index of the first threshold for which the sensitivity > tpr_cut
idx5 = np.min(np.where(tpr[0] > tpr_cut)) 
# index of the first threshold for which the fall out < fpr_cut
idx6 = np.max(np.where(fpr[0] < fpr_cut)) 

'''
print("Micro Average tpr > %.2f: %.4f" % (tpr_cut,thr["micro"][idx1]))
print("Micro Average fpr < %.2f: %.4f" % (fpr_cut,thr["micro"][idx2]))
print("Macro Average tpr > %.2f: %.4f" % (tpr_cut,thr["macro"][idx3]))
print("Macro Average fpr < %.2f: %.4f" % (fpr_cut,thr["macro"][idx4]))
'''
print("toxic label tpr > %.2f: %.4f" % (tpr_cut,thr[0][idx5]))
print("toxic label fpr < %.2f: %.4f" % (fpr_cut,thr[0][idx6]))

# Plot all ROC curves
import matplotlib.pyplot as plt
%matplotlib inline
from itertools import cycle

plt.figure(figsize=(10,8))
lw = 2

plt.figure(figsize=(10,8))
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.4f})'
               ''.format(rocauc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.4f})'
               ''.format(rocauc["macro"]),
         color='navy', linestyle=':', linewidth=4)
'''
plt.plot([0,fpr["micro"][idx1]], [tpr["micro"][idx1],tpr["micro"][idx1]], 'k--', color='blue')
plt.plot([fpr["micro"][idx1],fpr["micro"][idx1]], [0,tpr["micro"][idx1]], 'k--', color='blue')

plt.plot([0,fpr["micro"][idx2]], [tpr["micro"][idx2],tpr["micro"][idx2]], 'k--', color='red')
plt.plot([fpr["micro"][idx2],fpr["micro"][idx2]], [0,tpr["micro"][idx2]], 'k--', color='red')

plt.plot([0,fpr["macro"][idx3]], [tpr["macro"][idx3],tpr["macro"][idx3]], 'k--', color='blue')
plt.plot([fpr["macro"][idx3],fpr["macro"][idx3]], [0,tpr["macro"][idx3]], 'k--', color='blue')

plt.plot([0,fpr["macro"][idx4]], [tpr["macro"][idx4],tpr["macro"][idx4]], 'k--', color='red')
plt.plot([fpr["macro"][idx4],fpr["macro"][idx4]], [0,tpr["macro"][idx4]], 'k--', color='red')
'''
plt.plot([0,fpr[0][idx5]], [tpr[0][idx5],tpr[0][idx5]], 'k--', color='blue')
plt.plot([fpr[0][idx5],fpr[0][idx5]], [0,tpr[0][idx5]], 'k--', color='blue')

plt.plot([0,fpr[0][idx6]], [tpr[0][idx6],tpr[0][idx6]], 'k--', color='red')
plt.plot([fpr[0][idx6],fpr[0][idx6]], [0,tpr[0][idx6]], 'k--', color='red')

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
for i, color in zip(range(y_test.shape[1]), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of {0} (area = {1:0.4f})'
             ''.format(label_names[i], rocauc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves and Areas under the Curves')
plt.legend(loc="lower right")
plt.show()

In [17]:
# Visualize training history
import matplotlib.pyplot as plt
import numpy

# list all data in history
print(h.history.keys())

In [18]:


# Plot history for roc_auc scores
plt.figure(figsize=(10,8))
#plt.plot(h.history['tf_binary_auc'])
#plt.plot(h.history['val_tf_binary_auc'])
plt.plot(h.history['sklearn_auc'])
plt.plot(h.history['val_sklearn_auc'])
plt.title('model roc_auc vs epoch')
plt.ylabel('auc')
plt.xlabel('epoch')
#plt.legend(['tf binary auc train', 'tf binary auc test', 'sklearn auc train', 'sklearn auc test'], loc='upper left')
plt.legend(['sklearn auc train', 'sklearn auc validation'], loc='upper left')
plt.show()
# Plot history for loss
plt.figure(figsize=(10,8))
plt.plot(h.history['loss'])
plt.plot(h.history['val_loss'])
plt.title('model loss vs epoch')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()



In [19]:
# Save the prediction on test dataset
save_path = '/opt/data/share01/jl2408/pred_run5-234-tanh-25e-sd0.4-d0.2-adadelta-dot9830.csv'
pred_save = test.copy()
pred_save[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]] = y_pred
pred_save.to_csv(save_path, index=False)
pred_test = pd.read_csv(save_path)
pred_test = pred_test[label_names]
score = roc_auc_score(y_test, pred_test)
print("\n roc_auc score: %.6f \n" % (score))